## Named Entity Recognition

Named Entity Recognition (NER) is a fundamental Natural Language Processing (NLP) task that involves identifying and classifying named entities (like people, places, organizations) within text. For example, in the sentence "Shakespeare wrote Romeo and Juliet in London", a NER system would identify "Shakespeare" as a person, "Romeo and Juliet" as a work of art, and "London" as a location. NER is crucial for extracting structured information from unstructured text, making it valuable for tasks like information retrieval, question answering, and metadata enrichment. In this notebook, we'll explore how to perform NER using both traditional NLP approaches and modern Large Language Models.

## Rationale

This notebook demonstrates how to use OpenAI's GPT models to perform Named Entity Recognition (NER) by converting input text into annotated markdown format. Rather than using traditional NLP libraries, we leverage a Large Language Model's natural language understanding capabilities to identify and classify named entities. The notebook takes plain text as input and outputs markdown where entities are annotated in the format [Entity](TYPE), such as [London](LOCATION). This approach showcases how LLMs can be used for structured information extraction tasks in cultural heritage metadata enrichment.


## Process Overview

The process consists of the following steps:
1. **Text Input**: We start with plain text that needs entity recognition
2.  **LLM Processing**: The text is sent to GPT with a prompt that instructs it to identify entities
3. **Entity Annotation**: The LLM marks entities in markdown format: [Entity](TYPE)
4. **Visualization**: The annotated text is displayed with color-coded entity highlighting

This approach leverages the LLM's natural language understanding while producing structured, machine-readable output.

## Necessary Functions for Visualizations

In [24]:
import re
import spacy
from spacy.tokens import Doc, Span


def annotated_text_to_spacy_doc(text, nlp=None):
    """
    Converts annotated text in format [Entity](LABEL) to a spaCy Doc with entity spans.
    
    Args:
        text (str): Text with annotations like "[Tom](PERSON) worked for [Microsoft](ORGANIZATION)"
        nlp (spacy.Language, optional): spaCy language model. If None, uses blank English model.
    
    Returns:
        spacy.tokens.Doc: spaCy document with entity spans set
        
    Example:
        >>> text = "[Tom](PERSON) worked for [Microsoft](ORGANIZATION) in 2020 before he lived in [Rome](LOCATION)."
        >>> doc = annotated_text_to_spacy_doc(text)
        >>> spacy.displacy.render(doc, style="ent")
    """
    if nlp is None:
        nlp = spacy.blank("en")
    
    # Pattern to match [text](LABEL) format
    pattern = r'\[([^\]]+)\]\(([^)]+)\)'
    
    # Parse the text to extract tokens and entity information
    tokens = []
    entity_spans = []  # List of (start_token_idx, end_token_idx, label)
    custom_labels = set()
    
    # Split text by the pattern and process each part
    last_end = 0
    token_idx = 0
    
    for match in re.finditer(pattern, text):
        # Add tokens before the entity
        before_entity = text[last_end:match.start()]
        if before_entity.strip():
            # Tokenize the text before the entity
            before_tokens = before_entity.split()
            tokens.extend(before_tokens)
            token_idx += len(before_tokens)
        
        # Add the entity tokens
        entity_text = match.group(1)
        entity_label = match.group(2)
        custom_labels.add(entity_label)
        
        # Tokenize the entity text
        entity_tokens = entity_text.split()
        start_token_idx = token_idx
        tokens.extend(entity_tokens)
        token_idx += len(entity_tokens)
        end_token_idx = token_idx
        
        # Store entity span information
        entity_spans.append((start_token_idx, end_token_idx, entity_label))
        
        last_end = match.end()
    
    # Add any remaining tokens after the last entity
    remaining = text[last_end:]
    if remaining.strip():
        remaining_tokens = remaining.split()
        tokens.extend(remaining_tokens)
    
    # Add custom labels to the NLP model if they don't exist
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    for label in custom_labels:
        ner.add_label(label)
    
    # Create spaces array (True for tokens that should have a space after them)
    # Simple heuristic: all tokens except the last one get a space
    spaces = [True] * len(tokens)
    if tokens:
        spaces[-1] = False
    
    # Create the Doc from tokens
    doc = Doc(nlp.vocab, words=tokens, spaces=spaces)
    
    # Create entity spans
    entities = []
    for start_idx, end_idx, label in entity_spans:
        if start_idx < len(doc) and end_idx <= len(doc):
            span = Span(doc, start_idx, end_idx, label=label)
            entities.append(span)
    
    # Set entities on the document
    doc.ents = entities
    
    return doc


def visualize_annotated_text(text, nlp=None, style="ent", jupyter=True):
    """
    Convenience function to convert annotated text and visualize it with displaCy.
    
    Args:
        text (str): Text with annotations like "[Tom](PERSON) worked for [Microsoft](ORGANIZATION)"
        nlp (spacy.Language, optional): spaCy language model. If None, uses blank English model.
        style (str): displaCy style ("ent" or "dep")
        jupyter (bool): Whether to render for Jupyter notebook
    
    Returns:
        Rendered visualization (HTML string if not in Jupyter)
    """
    doc = annotated_text_to_spacy_doc(text, nlp)
    
    try:
        import spacy
        return spacy.displacy.render(doc, style=style, jupyter=jupyter)
    except ImportError:
        print("spaCy not installed. Please install with: pip install spacy")
        return None


## Importing the Required Libraries

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

## Loading our Environment Variables

In [14]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Connecting to OpenAI Client

In [15]:
client = OpenAI(api_key=OPENAI_API_KEY)

## Main Variables for the Notebook

In [16]:
MODEL = "gpt-4o-mini"
LABELS = ["PERSON", "LOCATION", "ORGANIZATION"]
TEXT = "They marched from Alexandria through Memphis via the Nile to Thebes."

## Creating the Prompt

In [17]:
prompt = f"""
Convert the following text into a structured markdown format, where you annotate the entities in the text in the following format: [Tom](PERSON) went to [New York](PLACE).

Look for the following entities types:
{LABELS}

Do this for the following text:
{TEXT}

Only return the markdown output, nothing else.
"""

In [18]:
print(prompt)


Convert the following text into a structured markdown format, where you annotate the entities in the text in the following format: [Tom](PERSON) went to [New York](PLACE).

Look for the following entities types:
['PERSON', 'LOCATION', 'ORGANIZATION']

Do this for the following text:
They marched from Alexandria through Memphis via the Nile to Thebes.

Only return the markdown output, nothing else.



## Calling OpenAI

In [19]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}]
)

In [20]:
markdown_output = response.choices[0].message.content

In [21]:
print(markdown_output)

They marched from [Alexandria](LOCATION) through [Memphis](LOCATION) via the [Nile](LOCATION) to [Thebes](LOCATION).


## Visualizing the Results

In [22]:
visualize_annotated_text(markdown_output)